# MODIS land cover exploration

Visualisation and stats generation

In [2]:
import ee

In [3]:
ee.Initialize()

In [5]:
import ipyleaflet

# utility
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

MODIS land cover data, `ee.ImageColelction` with 17 `ee.Image`, each representing a year (data between 2000 and 2016). Each year image has multiple bands, representing different land cover classifications

In [6]:
modis = ee.ImageCollection('MODIS/006/MCD12Q1')

In [31]:
meta_modis = modis.getInfo()

In [7]:
def get_modis_lc_by_year(year):
    
    if year not in range(2000, 2017):
        raise Exception('Year must be in between 2000 and 2016')
        
    # internal ee.Image 'system:time_start' 'system:time_end' in miliseconds
    g = modis.filterDate(str(year)+'-01-01', str(year)+'-12-31').first()
    
    return g

In [24]:
lc2015 = get_modis_lc_by_year(2015)

In [25]:
lc2015

Here `g` is an instance of `ee.Image`. Get an descriptive object `meta` to check on properties. Examine if the time filter works as intended. `meta` will also be used to derive colour schemes and etc. 

**N.B.**
For viz purposes, `meta` is the same regardless of the band/different LC classification within `modis`. This can be re-used and there is no need to repeatedly request `meta` from Earth Engine, based on the band

In [26]:
meta = lc2015.getInfo()

In [14]:
meta['properties']['system:time_start']

1420070400000

In [17]:
import datetime

In [19]:
datetime.datetime.fromtimestamp(meta['properties']['system:time_start']/1000.0)

datetime.datetime(2015, 1, 1, 0, 0)

In [20]:
datetime.datetime.fromtimestamp(meta['properties']['system:time_end']/1000.0)

datetime.datetime(2016, 1, 1, 0, 0)

Here are the properties of the `meta`, which is a dictionary

In [21]:
meta['properties'].keys()

[u'system:index',
 u'LC_Prop1_class_values',
 u'system:time_start',
 u'LC_Type3_class_values',
 u'LC_Type4_class_palette',
 u'system:footprint',
 u'LC_Prop2_class_names',
 u'LC_Type4_class_names',
 u'LW_class_palette',
 u'LC_Type5_class_values',
 u'LC_Prop2_class_values',
 u'LC_Type4_class_values',
 u'LC_Type5_class_names',
 u'LC_Type1_class_values',
 u'LC_Type2_class_palette',
 u'LC_Prop1_class_names',
 u'LC_Type1_class_names',
 u'LW_class_values',
 u'system:time_end',
 u'LC_Type3_class_palette',
 u'LC_Prop3_class_names',
 u'LC_Type2_class_values',
 u'LC_Prop3_class_palette',
 u'LC_Type2_class_names',
 u'LC_Prop1_class_palette',
 u'LC_Prop3_class_values',
 u'LC_Type3_class_names',
 u'system:asset_size',
 u'LC_Type1_class_palette',
 u'LC_Type5_class_palette',
 u'LW_class_names',
 u'LC_Prop2_class_palette']

## Visualisation

In [5]:
import ipyleaflet

# utility
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

In [5]:
import ipyleaflet

# utility
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

In [23]:
# function to return viz params, to be unpacked for use
def viz_setting(band):
    '''e.g. band="LC_Type1" '''
    values = meta['properties'][band+'_class_values']
    palette = meta['properties'][band+'_class_palette']
    
    return {'min': min(values),
           'max': max(values),
           'palette': palette,
           'bands': [band]}

# convenient wrapper
def viz_map(ee_image, band):
    map1 = ipyleaflet.Map(
    center=(48.2082, 16.3779), zoom=4,
    layout={'height':'400px'}
    )

    map1.add_layer(
        ipyleaflet.TileLayer(url=GetTileLayerUrl(
            # returns ee_image object
            ee_image.visualize(**viz_setting(band))
        )
    ))

    return map1

In [22]:
from IPython.core.display import display, HTML

# legend viz
def viz_legend(band):
    '''Create a legend based on band'''
    values = meta['properties'][band+'_class_values']
    palette = meta['properties'][band+'_class_palette']
    names = meta['properties'][band+'_class_names']
    
    html = ""
    for value, name, colour in zip(values, names, palette):
        html += '''
        <p style="font-size:12px">
        <div style="background-color:#{};display:inline-block;width:30px;height:12px">
        </div> {}: {}</p>
        '''.format(colour, value, name)
        
#     print html
    display(HTML(html))

## Examples

Below are examples of all LC classification within `modis` in year 2015

In [32]:
# description of the modis
display(HTML(meta_modis['properties']['description']))

Name,Units,Min,Max,Description
LC_Type1,,,,Land Cover Type 1: Annual International Geosphere-Biosphere Programme (IGBP) classification
LC_Type2,,,,Land Cover Type 2: Annual University of Maryland (UMD) classification
LC_Type3,,,,Land Cover Type 3: Annual Leaf Area Index (LAI) classification
LC_Type4,,,,Land Cover Type 4: Annual BIOME-Biogeochemical Cycles (BGC) classification
LC_Type5,,,,Land Cover Type 5: Annual Plant Functional Types classification
LC_Prop1_Assessment,%,0,100,LCCS1 land cover layer confidence
LC_Prop2_Assessment,%,0,100,LCCS2 land use layer confidence
LC_Prop3_Assessment,%,0,100,LCCS3 surface hydrology layer confidence
LC_Prop1,,,,FAO-Land Cover Classification System 1 (LCCS1) land cover layer
LC_Prop2,,,,FAO-LCCS2 land use layer


In [29]:
# Land Cover Type 1: Annual International Geosphere-Biosphere Programme (IGBP) classification
viz_map(lc2015, 'LC_Type1')

TWFwKGJhc2VtYXA9eyd1cmwnOiAnaHR0cHM6Ly97c30udGlsZS5vcGVuc3RyZWV0bWFwLm9yZy97en0ve3h9L3t5fS5wbmcnLCAnbWF4X3pvb20nOiAxOSwgJ2F0dHJpYnV0aW9uJzogJ01hcCDigKY=


In [30]:
viz_legend('LC_Type1')

In [33]:
# Land Cover Type 2: Annual University of Maryland (UMD) classification
viz_map(lc2015, 'LC_Type2')

TWFwKGJhc2VtYXA9eyd1cmwnOiAnaHR0cHM6Ly97c30udGlsZS5vcGVuc3RyZWV0bWFwLm9yZy97en0ve3h9L3t5fS5wbmcnLCAnbWF4X3pvb20nOiAxOSwgJ2F0dHJpYnV0aW9uJzogJ01hcCDigKY=


In [34]:
viz_legend('LC_Type2')

In [35]:
# Land Cover Type 3: Annual Leaf Area Index (LAI) classification
viz_map(lc2015, 'LC_Type3')

TWFwKGJhc2VtYXA9eyd1cmwnOiAnaHR0cHM6Ly97c30udGlsZS5vcGVuc3RyZWV0bWFwLm9yZy97en0ve3h9L3t5fS5wbmcnLCAnbWF4X3pvb20nOiAxOSwgJ2F0dHJpYnV0aW9uJzogJ01hcCDigKY=


In [36]:
viz_legend('LC_Type3')